# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
   
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
# TO-DO: Create a Keyspace 
KEYSPACE = 'events'
session.execute("""
CREATE KEYSPACE IF NOT EXISTS %s 
WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
""" %KEYSPACE)

#### Set Keyspace

In [11]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace(KEYSPACE)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Create table

In [67]:
table_1 = 'session_item'
session.execute('DROP TABLE IF EXISTS %s' %table_1)
query_1 = 'SELECT artist, song FROM %s where sessionId = 338 AND itemInSession = 4' %table_1
create_table_1 = 'CREATE TABLE IF NOT EXISTS %s (artist text, song text, length float,\
                  sessionId int, itemInSession int,\
                  PRIMARY KEY (sessionId, itemInSession))' % table_1
session.execute(create_table_1)

#### Insert data to table

In [68]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item (artist, song , length, sessionId, itemInSession)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Check data insetion

In [116]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows=session.execute('SELECT * FROM %s limit 5' %table_1)
for row in rows:
    print(row)

Row(sessionid=23, iteminsession=0, song='The Calculation (Album Version)', artist='Regina Spektor', length=191.08526611328125)
Row(sessionid=23, iteminsession=1, song='All Of The Champs That Ever Lived', artist='Octopus Project', length=250.95791625976562)
Row(sessionid=23, iteminsession=2, song='So Jealous', artist='Tegan And Sara', length=180.06158447265625)
Row(sessionid=23, iteminsession=3, song='Okay Dolores', artist='Dragonette', length=153.39056396484375)
Row(sessionid=23, iteminsession=4, song='Drop The World', artist='Lil Wayne / Eminem', length=229.58975219726562)


#### Test query

In [117]:
rows=session.execute(query_1)
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)')


In [173]:
events_df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,A Fine Frenzy,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",256,Almost Lover (Album Version),69
1,Nirvana,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",237,Serve The Servants,44
2,Television,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",237,See No Evil (Remastered LP Version),44
3,JOHN COLTRANE,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",237,Blues To Bechet (LP Version),44
4,NOFX,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",237,It's My Job To Keep Punk Rock Elite,44


### Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create table

In [192]:
table_2 = 'artist_song_user'
query_2 = 'SELECT artist, song, firstname, lastname FROM %s WHERE userid = 10 AND sessionid = 182 ALLOW FILTERING' %table_2
session.execute('DROP TABLE IF EXISTS %s' %table_2)
create_table_2 = 'CREATE TABLE IF NOT EXISTS %s (artist text, song text, firstName text,\
                  lastName text, userid int, sessionid int, iteminsession int, \
                  PRIMARY KEY (iteminsession, userid, sessionid))' % table_2
session.execute(create_table_2)

#### Insert data to table

In [193]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (artist, song , firstname, lastname, userid, sessionid, iteminsession)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

#### Check data insertion

In [194]:
rows = session.execute('SELECT * FROM %s LIMIT 5' %table_2)
for row in rows:
    print(row)

Row(iteminsession=23, userid=15, sessionid=172, artist='She & Him', firstname='Lily', lastname='Koch', song='Got Me')
Row(iteminsession=23, userid=15, sessionid=199, artist='Juanes', firstname='Lily', lastname='Koch', song='No Siento Penas')
Row(iteminsession=23, userid=15, sessionid=221, artist='Fall Out Boy', firstname='Lily', lastname='Koch', song='Nobody Puts Baby In The Corner')
Row(iteminsession=23, userid=15, sessionid=716, artist='Madeleine Peyroux', firstname='Lily', lastname='Koch', song='Instead')
Row(iteminsession=23, userid=15, sessionid=818, artist='Al Green', firstname='Lily', lastname='Koch', song='I Say A Little Prayer')


#### Check the result

In [203]:
rows = session.execute(query_2)
for row in rows:
    print(row)

Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


#### Create table

In [214]:
table_3 = 'user_song'
query_3 = "SELECT firstname, lastname FROM %s WHERE song = 'All Hands Against His Own'" %table_3
session.execute('DROP TABLE IF EXISTS %s' %table_3)
create_table_3 = 'CREATE TABLE IF NOT EXISTS %s (userid int, firstname text, lastname text, song text,\
                  PRIMARY KEY (song, userid))' % table_3
session.execute(create_table_3)

#### Insert data into table

In [215]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (userid, firstname, lastname, song)"
        query = query + " VALUES(%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

#### Check data insertion

In [216]:
rows = session.execute('SELECT * FROM %s LIMIT 5'%table_3)
for row in rows:
    print(row)

Row(song="Wonder What's Next", userid=49, firstname='Chloe', lastname='Cuevas')
Row(song="In The Dragon's Den", userid=49, firstname='Chloe', lastname='Cuevas')
Row(song='Too Tough (1994 Digital Remaster)', userid=44, firstname='Aleena', lastname='Kirby')
Row(song='Rio De Janeiro Blue (Album Version)', userid=49, firstname='Chloe', lastname='Cuevas')
Row(song='My Place', userid=15, firstname='Lily', lastname='Koch')


#### Validate target query

In [218]:
rows = session.execute(query_3)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [221]:
session.execute('DROP TABLE IF EXISTS %s' %table_1)
session.execute('DROP TABLE IF EXISTS %s' %table_2)
session.execute('DROP TABLE IF EXISTS %s' %table_3)

### Close the session and cluster connection¶

In [222]:
session.shutdown()
cluster.shutdown()